# Big Data Modeling and Management Assigment - Homework 1

# Submission

GROUP NUMBER: **1** 

GROUP MEMBERS:

|STUDENT NAME|STUDENT NUMBER|
|---|---|
|Afonso Ascensão|20240684|
|André Filipe Silva|20230972|
|Joana Esteves|20240746|
|Susana Reis|20240567|
|Tomás Figueiredo|20240941|

## 🍺 The Beer project  🍺 

As it was shown in classes, graph databases are a natural way of navegating related information. For this first project we will be taking a graph database to analyse beer and breweries!   

The project datasets are based on [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. 

### Problem description

Imagine you are working in the Data Management department of Analytics company.
Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions while adjusting the database to meet the needs of your colleagues.
Please record and keep track of your database changes, and submit the file with all cells run and with the output shown.

### Questions

1. Explore the database: get familiar with current schema, elements and other important database parameters. [1 point]
2. Adjust the database and mention reasoning behind: e.g. clean errors, remove redundancies, adjust schema as necessary. Visualize the final version of database schema. [4 points]
3. Analytics department requires the following information for the biweekly reporting: [5 points]
    1. How many reviews has the beer with the most reviews?
    2. Which three users wrote the most reviews about beers?
    3. Find all beers that are described with following words: 'fruit', 'complex', 'nutty', 'dark'.
    4. Which top three breweries produce the largest variety of beer styles?
    5. Which country produces the most beer styles?
4. Market Analysis department in your company accesses and updates the trends data on the daily basis. Given that, consider how you need to optimize the database and its performance so that the following queries are efficient. Measure performance to communicate your improvements using PROFILE before final query. Answer the following: [4 points]
    1. Using ABV score, find five strongest beers, display their ABV score and the corresponding brewery? Keep in mind that the strongest known beer is Snake Venom, and deal with the error entries in the database.
    2. Using the answer from question 2, find the top 5 distict beer styles with the highest average score of smell + feel that were reviewed by the third most productive user. Keep in mind that cleaning the database earlier should ensure correct results.
5. Answer **two out of four** of the following questions using Graph Algorithms (gds): [NB: make sure to clear the graph before using it again] For the quarterly report, Analytics department the follownig information. [6 points]
    1. Which two countries are most similiar when it comes to their top five most produced Beer styles?
    2. Which beer is the most popular when considering the number of users who reviewed it? 
    3. Users are connected together by their reviews of beers, taking into consideration the "smell" score they assign as a weight, how many communities are formed from these relationships? How many users are in the three largest communities? 
    4. Which user is the most influential when it comes to reviews of distinct beers by style?
 
### Groups  

Groups should have 4 people maximum. Please mark which group you are here: https://shorturl.at/zE0QP 

### Submission      

The code used to produce the results and to-the-point explations should be uploaded to moodle. They should have a clear reference to the group, either on the file name or on the document itself. Preferably one Jupyter notebook per group.

Delivery date: Until the **midnight of March 18, 2025**

### Evaluation   

This will be 20% of the final grade.   
Each solution will be evaluated on 2 components: correctness of results and efficiency of the query (based on database schema).  
All code will go through plagiarism automated checks. Groups with the same code will undergo investigation.

In [ ]:
from neo4j import GraphDatabase
from pprint import pprint
from py2neo import Graph
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
NEO4J_URI="neo4j://localhost:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="test"

In [ ]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD), )

# Functions to run queries in Neo4J

In [ ]:
def execute_write(driver, query):
    with driver.session(database="neo4j") as session:
        # Write transactions allow the driver to handle retries and transient errors
        result = session.execute_write(lambda tx, query: list(tx.run(query)), query)
    return result

In [ ]:
def execute_read(driver, query):    
    with driver.session(database="neo4j") as session:
        result = session.execute_read(lambda tx, query: list(tx.run(query)), query)
    return result

# 1. Explore the database

## 1.1 Explore schema:

In [ ]:
# Get "visualization" schema. This generates the visualization schema and is better ran using the browser, but we put the query here for completion.

query = """
    call db.schema.visualization;
    """

result = execute_read(driver, query)

pprint(result)

In [ ]:
## Below is code for plotting the database statically in the notebook. Needs fix to include relationships and not only nodes. Will look at it later.
# This consumes quite a lot of memory so it is commented out for now

In [ ]:
# graph = Graph("bolt://localhost:7687", auth=("neo4j", "test"))

In [ ]:
# # Fetch schema visualization data
# query = "CALL db.schema.visualization()"
# results = graph.run(query).data()

# # Extract graph schema
# G = nx.Graph()

# # Add nodes (labels)
# for node in results[0]['nodes']:
#     label = node.get('name')  # Ensure we get the node label
#     if label:
#         G.add_node(label)

# # Add relationships
# for rel in results[0]['relationships']:
#     start = rel.get('startNode')
#     end = rel.get('endNode')
#     rel_type = rel.get('type', 'rel')  # Default relationship type

#     if start and end:  # Avoid None values
#         G.add_edge(start, end, label=rel_type)

# # Draw the graph
# plt.figure(figsize=(8, 6))
# pos = nx.spring_layout(G)  # Layout for visualization
# nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray', node_size=2000, font_size=10)

# plt.title("Neo4j Schema Visualization")
# plt.show()

In [ ]:
# Get number of observations 

query = """
        MATCH () RETURN count(*)
    """

result = execute_read(driver, query)

print(f"We have {result[0][0]} observations in the database.")

In [ ]:
# Check indexes

query = """
        call db.indexes;
    """

result = execute_read(driver, query)

pprint(result)

Indexes:
- Style, user, countries and cities id is the property name.
- Reviews, Brewries, Beers id id the property id.
- We have to make sure these properties have unique values.

In [ ]:
# Check constraints

query = """
        call db.constraints;
    """

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check all labels

query = """
        call db.labels;
    """

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Retrieve all nodes in the database
# group them by label and count how many nodes exist for each label

query = """
        MATCH (n)
        RETURN labels(n) AS node_label, count(n) AS node_count
        ORDER BY node_count DESC
    """

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check all types of relationships

query = """
        call db.relationshipTypes();
    """

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check all properties of the BEERS node
query = """
MATCH (n:BEERS) 
RETURN n, keys(n) AS propertyKeys
LIMIT 1
"""

result = execute_read(driver, query)
pprint(result[0]["propertyKeys"])

In [ ]:
# Check all properties of the BREWERIES node
query = """
MATCH (n:BREWERIES) 
RETURN n, keys(n) AS propertyKeys
LIMIT 1
"""

result = execute_read(driver, query)
pprint(result[0]["propertyKeys"])

In [ ]:
# Check all properties of the USER node
query = """
MATCH (n:USER) 
RETURN n, keys(n) AS propertyKeys
LIMIT 1
"""

result = execute_read(driver, query)
pprint(result[0]["propertyKeys"])

In [ ]:
# Check all properties of the REVIEWS node
query = """
MATCH (n:REVIEWS) 
RETURN n, keys(n) AS propertyKeys
LIMIT 1
"""

result = execute_read(driver, query)
pprint(result[0]["propertyKeys"])

In [ ]:
# Check all properties of the CITIES node
query = """
MATCH (n:CITIES) 
RETURN n, keys(n) AS propertyKeys
LIMIT 1
"""

result = execute_read(driver, query)
pprint(result[0]["propertyKeys"])

In [ ]:
# Check all properties of the COUNTRIES node
query = """
MATCH (n:COUNTRIES) 
RETURN n, keys(n) AS propertyKeys
LIMIT 1
"""

result = execute_read(driver, query)
pprint(result[0]["propertyKeys"])

In [ ]:
# Check all properties of the STYLE node
query = """
MATCH (n:STYLE) 
RETURN n, keys(n) AS propertyKeys
LIMIT 1
"""

result = execute_read(driver, query)
pprint(result[0]["propertyKeys"])

In [ ]:
# Retrieve all unique relationship patterns in the database 
# Return the labels of the starting node, the type of relationship, and the labels of the ending node
# Not specifying direction

query = """
        MATCH (a)-[r]-(b)
        RETURN DISTINCT 
        labels(a) AS start_node_labels, 
        type(r) AS relationship_type, 
        labels(b) AS end_node_labels
    """

result = execute_read(driver, query)

pprint(result)


- All relationships are bidirectional. 
- It may make more sense for relationships to be unidirectional, because for example:
    - A brewery brews beer, but a beer does not brew breweries. 
- Analyze if we should change relationships to be unidirectional.
- Suggestions: 
    - (USERS)-[POSTED]->(REVIEWS)
    - (CITIES)-[IN]->(COUNTRIES)
    - (BREWERIES)-[IN]->(CITIES)
    - (BREWERIES)-[BREWS]->(BEER)
    - (BEERS)->[HAS_STYLE]->(STYLE)
    - (REVIEWS)->[REVIEWED]->(BEERS)
    - (BEERS)->[HAS_STYLE]->(STYLE)


**André** - A query abaixo está a retornar duplicados acho eu. acima fiz individualmente para cada node,  as properties de cada um

In [ ]:
# # Check properties for each node 

# query = """
#         MATCH (n)
#         RETURN DISTINCT labels(n) AS node_label, keys(n) AS node_properties
#     """

# result = execute_read(driver, query)

# pprint(result)

In [ ]:
# Check properties for each relationship

query = """
        MATCH ()-[r]->()
        RETURN DISTINCT type(r) AS relationship_type, keys(r) AS relationship_properties
    """

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check for isolated nodes

query = """
        MATCH (n)
        WHERE NOT EXISTS ((n)--())
        RETURN n
    """

result = execute_read(driver, query)

pprint(result)

- There are several nodes with the label REVIEWS that don't have any relationship (isolated nodes).

In [ ]:
# Check if there are any nodes w/ label REVIEWS that don't have any property
query = """
    MATCH (re:REVIEWS)
WHERE size(keys(re)) = 0
RETURN re
"""
result = execute_read(driver, query)

pprint(result)

- There are several reviews that have no properties.

## 1.2 Check for inconsistencies: 

In [ ]:
# Check for duplicated nodes w/ label COUNTRIES and same name

query = """
    MATCH (co:COUNTRIES)
    WITH co.name AS countryName, count(co) AS countryCount
    WHERE countryCount > 1
    RETURN countryName, countryCount
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check for duplicated nodes w/ label CITIES and same name

query = """
    MATCH (ci:CITIES)
    WITH ci.name AS cityName, count(ci) AS cityCount
    WHERE cityCount > 1
    RETURN cityName, cityCount
"""

result = execute_read(driver, query)

pprint(result)

- Since that the only property for COUNTRIES, CITIES, STYLE and USER is name, having more than one node with the same name represents a duplicated node.
- There are several duplicated nodes for both COUNTRIES and CITIES labels.

In [ ]:
# Check for duplicated nodes w/ label STYLE and same name

query = """
    MATCH (st:STYLE)
    WITH st.name AS styleName,
    count(st) AS styleCount
    WHERE styleCount > 1
    RETURN styleName, styleCount
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check for duplicated nodes w/ label USER and same name

query = """
    MATCH (us:USER)
    WITH us.name AS userName,
    count(us) AS userCount
    WHERE userCount > 1
    RETURN userName, userCount
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check for duplicated nodes w/ label BREWERY 

query = """
    MATCH (br:BREWERY)
    WITH br.id AS breweryId, 
        br.state AS breweryState,
        br.notes AS breweryNotes,
        br.types AS breweryTypes,
        br.name AS breweryName,
    count(br) AS breweryCount
    WHERE breweryCount > 1
    RETURN breweryId, breweryName, breweryCount
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check for duplicated nodes w/ label BREWERY and same id

query = """
    MATCH (br:BREWERY)
    WITH br.id AS breweryId, 
    count(br) AS breweryCount
    WHERE breweryCount > 1
    RETURN breweryId, breweryCount
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check for duplicated nodes w/ label BREWERY and same name

query = """
    MATCH (br:BREWERY)
    WITH br.name AS breweryName,
    count(br) AS breweryCount
    WHERE breweryCount > 1
    RETURN breweryName, breweryCount
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check for duplicated nodes w/ label BEERS 

query = """
    MATCH (be:BEERS)
    WITH be.id AS beerId, 
        be.retired AS beerRetired, 
        be.availability AS beerAvailability, 
        be.brewery_id AS beerBreId, 
        be.state AS beerState, 
        be.name AS beerName, 
        be.abv AS beerAbv, 
        be.notes AS beerNotes, 
    count(be) AS beerCount
    WHERE beerCount > 1
    RETURN beerId, beerName, beerCount
"""

result = execute_read(driver, query)

pprint(result)

- There are several nodes with the label BEERS duplicated for all properties.

In [ ]:
# Check for duplicated nodes w/ label REVIEWS 

query = """
    MATCH (re:REVIEWS)
    WITH re.text AS reviewTest,
        re.beer_id AS reviewBeerId,
        re.smell AS reviewSmell,
        re.look AS reviewLook,
        re.score AS reviewScore,
        re.taste AS reviewTaste,
        re.feel AS reviewFeel,
        re.overall AS reviewOverall,
        re.id AS reviewId,
        re.date AS reviewDate,
    count(re) AS reviewCount
    WHERE reviewCount > 1
    RETURN reviewId, reviewCount
"""

result = execute_read(driver, query)

pprint(result)

- There are several nodes with the label REVIEWS duplicated for all properties.

Considerations:
- One city should only be associated w/ one country.
- One brewery should only be associated w/ one city.
- One review should only be associated w/ one user.
- One review should only be associated w/ one beer.
- One beer should only be associated w/ one style? -- **Andre: Uma beer nao pode ter mais de um estilo?**
- One beer should only be associated w/ one brewery? -- **Andre: Nao necessariamente? uma beer pode ser feita em diferentes breweries talvez?** 

In [ ]:
# Check if there is any brewery associated with more than one city
# Retrieve all brewery to city relationships
# In these relationships, count nº of cities for each brewery 

query = """
    MATCH (br:BREWERIES)-[:IN]->(ci:CITIES)
    WITH br, count(ci) AS city_count
    WHERE city_count > 1
    RETURN br.name, city_count
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
# Check if there is any city associated with more than one country
# Retrieve all city to country relationships
# In these relationships, count nº of countries for each city 

query = """
    MATCH (ci:CITY)-[:IN]->(co:COUNTRIES)
    WITH ci, count(co) AS country_count
    WHERE country_count > 1
    RETURN ci.name, country_count
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
query = """
    MATCH (us:USER)-[:POSTED]->(re:REVIEW)
    WITH re, count(us) AS user_count
    WHERE user_count > 1
    RETURN re.id, user_count
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
query = """
    MATCH (re:REVIEW)-[:REVIEWD]->(be:BEERS)
    WITH re, count(be) AS beer_count
    WHERE beer_count > 1
    RETURN re.id, beer_count
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
query = """
    MATCH (re:BEERS)-[:HAS_STYLE]->(st:STYLES)
    WITH re, count(st) AS style_count
    WHERE style_count > 1
    RETURN re.name, style_count
"""

result = execute_read(driver, query)

pprint(result)

In [ ]:
query = """
    MATCH (be:BEERS)-[:BREWED]->(br:BREWERIES)
    WITH be, COUNT(br) AS brewery_count
    WHERE brewery_count > 1
    RETURN be.name, brewery_count
"""

result = execute_read(driver, query)

pprint(result)

Considerations:
- All breweries should be associated with a city.
- All cities should be associated with a country.
- All reviews should be associated to a user.
- All reviews shoul be associated with a beer. 
- All beers should be associated with a brewery. 



In [ ]:
# Check if there is any is any node w/ BREWERIES label
# that does not have the specified relationship to a node w/ label CITIES

query = """
    MATCH (br:BREWERIES)
    WHERE NOT EXISTS ((br)-[:IN]->(:CITIES))
    RETURN br
"""
result = execute_read(driver, query)

pprint(result)

In [ ]:
query = """
    MATCH (ci:CITIES)
    WHERE NOT EXISTS ((ci)-[:IN]->(:COUNTRIES))
    RETURN ci
"""
result = execute_read(driver, query)

pprint(result)

In [ ]:
query = """
    MATCH (re:REVIEWS)
    WHERE NOT EXISTS ((re)-[:POSTED]->(:USER))
    RETURN re
"""
result = execute_read(driver, query)

pprint(result)

In [ ]:
query = """
    MATCH (re:REVIEWS)
    WHERE NOT EXISTS ((re)-[:REVIEWED]->(:BEERS))
    RETURN re
"""
result = execute_read(driver, query)

pprint(result)

In [ ]:
query = """
    MATCH (br:BREWERY)
    WHERE NOT EXISTS ((br)-[:BREWES]->(:BEERS))
    RETURN br
"""
result = execute_read(driver, query)

pprint(result)

- There are breweries not associated w/ any city.
- There are cities not associated w/ any country.
- There are reviews not associated w/ any user or beer.
- There are beers not associated w/ any brewery.

## 1.3 EXERCISE SUMMARY
- Indexes: Style, user, countries and cities id is the property name.Reviews, Breweries, Beers id id the property id.
- Labels: COUNTRIES, CITIES, BREWERIES, BEERS, REVIEWS, STYLE and USER.

- Relationships:
    - Relationship names: REVIEWED, BREWED, IN, HAS_STYLE and POSTED.
    - Relationships are all bidirectional, in all cases cases it seems more appropriate to have unidirectional rel.
    - Relationships have no properties. 
    - There are several nodes with the label REVIEWS that don't have any relationship (isolated nodes).

- Properties:
    - COUNTRIES, CITIES, USER and STYLE only property: ‘name’.
    - REVIEWS properties: 'text', 'smell', 'look', 'taste', 'feel', 'overall', 'beer_id', 'id', 'date', 'score'
    - There are several node w/ label REVIEWS that have no properties.
    - BREWERIES properties: 'notes', 'types', 'id', 'name', 'state'
    - BEERS properties: notes', 'abv', 'name', 'state', 'id', 'retired', 'availability', 'brewery_id'

- Inconsistencies found:
    - Duplicated nodes: There are several nodes with the label BEERS, COUNTRIES, CITIES and REVIEWS duplicated for all properties.
    - There are breweries not associated w/ any city.
    - There are cities not associated w/ any country.
    - There are reviews not associated w/ any user or beer.
    - There are beers not associated w/ any brewery.

# 2. Adjust the database and mention reasoning behind: e.g. clean errors, remove redundancies, adjust schema as necessary. Visualize the final version of database schema.

In [ ]:
# The first query below is not efficient. Try the second one. I still haven't tried it -- Andre

In [ ]:
# ## Changing from bidirectional relationships to unidirectional where it makes sense

# query = """
# MATCH (a)-[r]->(b)
# WHERE (type(r) = "IN" AND labels(a) = ["CITIES"] AND labels(b) = ["COUNTRIES"])
#    OR (type(r) = "IN" AND labels(a) = ["BREWERIES"] AND labels(b) = ["CITIES"])
#    OR (type(r) = "BREWS" AND labels(a) = ["BREWERIES"] AND labels(b) = ["BEERS"])
#    OR (type(r) = "HAS_STYLE" AND labels(a) = ["BEERS"] AND labels(b) = ["STYLE"])
#    OR (type(r) = "REVIEWED" AND labels(a) = ["REVIEWS"] AND labels(b) = ["BEERS"])
#    OR (type(r) = "POSTED" AND labels(a) = ["USER"] AND labels(b) = ["REVIEWS"])
# DELETE r
# """

# result = execute_write(driver, query)
# pprint(result)

In [ ]:
## Changing from bidirectional relationships to unidirectional where it makes sense
## using batch processing for faster computation

query = """
CALL apoc.periodic.iterate(
  "MATCH (a)-[r1]->(b) MATCH (b)-[r2]->(a)
   WHERE type(r1) = type(r2) 
   AND (
       (type(r1) = 'IN' AND labels(a) = ['CITIES'] AND labels(b) = ['COUNTRIES']) OR
       (type(r1) = 'IN' AND labels(a) = ['BREWERIES'] AND labels(b) = ['CITIES']) OR
       (type(r1) = 'BREWS' AND labels(a) = ['BREWERIES'] AND labels(b) = ['BEERS']) OR
       (type(r1) = 'HAS_STYLE' AND labels(a) = ['BEERS'] AND labels(b) = ['STYLE']) OR
       (type(r1) = 'REVIEWED' AND labels(a) = ['REVIEWS'] AND labels(b) = ['BEERS']) OR
       (type(r1) = 'POSTED' AND labels(a) = ['USER'] AND labels(b) = ['REVIEWS'])
   )
   AND id(r1) > id(r2)
   RETURN r2",
  "DELETE r2",
  {batchSize:1000}
);
"""

result = execute_write(driver, query)
pprint(result)

In [ ]:
# Checking how it looks after the changes

query = """
        MATCH (a)-[r]-(b)
        RETURN DISTINCT 
        labels(a) AS start_node_labels, 
        type(r) AS relationship_type, 
        labels(b) AS end_node_labels
    """

result = execute_read(driver, query)

pprint(result)

### 3.1 How many reviews has the beer with the most reviews?

In [ ]:
query = """
        MATCH (b:BEERS)-[r]->(:REVIEWS)
        WITH b, count(r) AS review_count
        ORDER BY review_count DESC
        LIMIT 1
        RETURN b.name, review_count


    """

result = execute_read(driver, query)
pprint(result)


OR

In [ ]:
query = """
        MATCH (b:BEERS)-[r]->(:REVIEWS)
        WITH b.name AS beer_name, count(r) AS review_count
        ORDER BY review_count DESC
        LIMIT 1 
        RETURN beer_name, review_count

    """

result = execute_read(driver, query)
pprint(result)

### 3.2 Which three users wrote the most reviews about beers?

In [ ]:
query = """
        MATCH (r:REVIEWS)-[p:POSTED]->(u:USER)
        WITH u, count(r) AS review_count
        ORDER BY review_count DESC
        LIMIT 3
        RETURN u.name
    """

result = execute_read(driver, query)
pprint(result)

### *REVIEW* 3.3 Find all beers that are described with following words: 'fruit', 'complex', 'nutty', 'dark'. *REVIEW*

In [ ]:
words = ['fruit', 'complex', 'nutty', 'dark']

query = f"""
        WITH {words} AS words

        MATCH (u:BEERS)-[r:REVIEWED]->(rs:REVIEWS)
        

        WHERE any(word IN words WHERE rs.text CONTAINS word)

        RETURN DISTINCT u.name
    """

result = execute_read(driver, query)

pprint(result)

### 3.5 Which top three breweries produce the largest variety of beer styles?

In [ ]:
query = """
        MATCH (br:BREWERIES)-[:BREWED]->(b:BEERS)-[h:HAS_STYLE]->(s:STYLE)
        RETURN br.name, count(s)
        ORDER BY count(s) DESC
        LIMIT 3 
    """

result = execute_read(driver, query)
pprint(result)

### 3.5 Which country produces the most beer styles?

In [ ]:
query = """
        MATCH (c:COUNTRIES)<-[:IN]-(ci:CITIES)<-[:IN]-(br:BREWERIES)-[:BREWED]->(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
        RETURN c.name, count(s)
        ORDER BY count(s) DESC
        LIMIT 1
    """

result = execute_read(driver, query)
pprint(result)